# Part 1 -- Setup & Collection

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [1]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/ipynb


In [2]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

### Check who our oldest Twitter user is (for an idea of how far back to go):

In [3]:
!pip install pymongo tweepy
import pymongo
import tweepy

In [4]:
twitter_30_df = pd.read_csv('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/data/twitter_users_30.csv')
twitter_30_df.sample(3)

,Name,Handle,Followers,Description,Type,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
2,Bill Gates,BillGates,"34,900,000",Sharing things I'm learning through my foundat...,Celebrity,NaN,NaN,NaN,NaN
10,Mark Cuban,mcuban,"7,040,000",Tech investor,Celebrity,NaN,NaN,NaN,NaN
17,Paul Graham,paulg,"862,000",Y Combinator,Thought Leader,NaN,NaN,NaN,NaN


In [5]:
handle = list(twitter_30_df['Handle'])

In [6]:
# Replace the API_KEY and API_SECRET with your application's key and secret:
auth = tweepy.AppAuthHandler(my_keys['CONSUMER_KEY'], my_keys['CONSUMER_SECRET'])

# Authorize twitter, initialize tweepy
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [7]:
for h in handle:
    user_data = api.get_user(h)
    print(user_data.created_at, user_data.name)

(datetime.datetime(2007, 3, 5, 22, 8, 25), u'Barack Obama')
(datetime.datetime(2007, 1, 2, 1, 48, 14), u'CNN Breaking News')
(datetime.datetime(2009, 6, 24, 18, 44, 10), u'Bill Gates')
(datetime.datetime(2009, 3, 18, 13, 46, 38), u'Donald J. Trump')
(datetime.datetime(2007, 5, 12, 13, 4, 50), u'The Economist')
(datetime.datetime(2009, 1, 16, 7, 40, 6), u'ashton kutcher')
(datetime.datetime(2013, 4, 9, 18, 4, 35), u'Hillary Clinton')
(datetime.datetime(2007, 3, 7, 1, 27, 9), u'TechCrunch')
(datetime.datetime(2009, 6, 2, 20, 12, 29), u'Elon Musk')
(datetime.datetime(2008, 5, 6, 19, 36, 33), u'The New Yorker')
(datetime.datetime(2008, 9, 10, 21, 12, 1), u'Mark Cuban')
(datetime.datetime(2006, 3, 21, 20, 50, 14), u'jack')
(datetime.datetime(2007, 1, 11, 5, 15, 48), u'MarketWatch')
(datetime.datetime(2009, 2, 9, 0, 3, 41), u'CNBC')
(datetime.datetime(2008, 5, 23, 19, 32, 37), u'Forbes Tech News')
(datetime.datetime(2006, 7, 13, 9, 5, 49), u'Chris Sacca')
(datetime.datetime(2007, 9, 12, 21, 

In [8]:
cli = pymongo.MongoClient(host='35.163.253.8')

In [9]:
test_collection = cli.twitter_db.test_collection
cli.database_names()

[u'twitter_db']

In [32]:
# test_collection.drop()

In [10]:
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/tools/GetOldTweets-python-master')
!pip install -r requirements.txt
import got

  Using cached lxml-3.5.0.tar.gz
  Using cached pyquery-1.2.10-py2-none-any.whl
  Running setup.py bdist_wheel for lxml ... error
  Complete output from command /opt/conda/envs/python2/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-ZJnk2c/lxml/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpBKZhV7pip-wheel- --python-tag cp27:
  Building lxml version 3.5.0.
  Building without Cython.
  ERROR: /bin/sh: 1: xslt-config: not found
  
  ** make sure the development packages of libxml2 and libxslt are installed **
  
  Using build configuration of libxslt
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-2.7
  creating build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/sax.py -> build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/__init__.py -> build/lib.linux-x86_64-2.7/lxml
  copying src/lxml/builde

Command "/opt/conda/envs/python2/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-ZJnk2c/lxml/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-9JPmKy-record/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-build-ZJnk2c/lxml/


### Import lookup data (of the 30 thought leaders). Split into 5 lists (of 6 accounts) for distributed processing on seperate AWS T2-Micro instances.

In [15]:
def get_tweets_to_mongo(handle_lookup):
    tweet_list = []
    for i in handle_lookup:
        try:
            tweetCriteria = got.manager.TweetCriteria().setUsername(i).setSince("2015-04-01").setUntil("2017-04-01").setMaxTweets(1000)
            tweet = got.manager.TweetManager.getTweets(tweetCriteria)
            tweetCount = 0
            for t in tweet:
                tweet_dict = {
                    'username': t.username,
                    'text': t.text,
                    'timestamp': t.date
                }
                tweet_list.append(tweet_dict)
                tweetCount += 1
                if (len(tweet_list)+1)%100==True:
                    print("Downloaded {} tweets".format(len(tweet_list)))
                
        except ValueError as e:
            print "Error occurred: " + str(e)
            pass
        try:
            mongo_response = test_collection.insert_one(tweet_dict)
            print("Inserting {0} tweets to Mongo for user: {1}".format(tweetCount, i))
        except:
            pass

    print ("\nTotal tweets downloaded & stored in mongo: {}".format(len(tweet_list)))

In [16]:
from datetime import datetime
start = datetime.now()
get_tweets_to_mongo(handle)
end = datetime.now()
print(end - start)


Total tweets downloaded & stored in mongo: 0
0:01:59.138103


In [11]:
cli.twitter_db.collection_names()

[u'test_collection', u'incomplete_collection']

In [14]:
test_collection.count()

0

In [13]:
test_collection.drop()

In [70]:
curs = twitter_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

,_id,text,timestamp,username
0,592f37a6fa2557009e159319,"President Obama tells the story of ""Fired up! ...",2012-11-06 06:07:10,BarackObama


In [37]:
df.shape

(6, 4)